In [13]:
import pandas as pd

data = pd.read_csv("troop_movements10m.csv")
print(data.head())

             timestamp  unit_id     unit_type  location_x  location_y  \
0  2023-06-13 17:33:18        1         at-st         2.0         8.0   
1  2023-06-13 17:33:17        2  tie_silencer         4.0         4.0   
2  2023-06-13 17:33:16        3         at-at         0.0         3.0   
3  2023-06-13 17:33:15        4  tie_silencer         6.0         1.0   
4  2023-06-13 17:33:14        5   tie_fighter         0.0         4.0   

   destination_x  destination_y    homeworld  
0              1              1  Glee Anselm  
1              0              1    Trandosha  
2              6              1     Corellia  
3              6              9        Shili  
4              9              6   Muunilinst  


In [14]:
data["unit_type"] = data["unit_type"].apply(lambda x: "unknown" if x == "invalid_unit" else x)
data["location_x"] = data["location_x"].ffill()
data["location_y"] = data["location_y"].ffill()
print(data.head())

             timestamp  unit_id     unit_type  location_x  location_y  \
0  2023-06-13 17:33:18        1         at-st         2.0         8.0   
1  2023-06-13 17:33:17        2  tie_silencer         4.0         4.0   
2  2023-06-13 17:33:16        3         at-at         0.0         3.0   
3  2023-06-13 17:33:15        4  tie_silencer         6.0         1.0   
4  2023-06-13 17:33:14        5   tie_fighter         0.0         4.0   

   destination_x  destination_y    homeworld  
0              1              1  Glee Anselm  
1              0              1    Trandosha  
2              6              1     Corellia  
3              6              9        Shili  
4              9              6   Muunilinst  


In [18]:
! pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [20]:
import pyarrow as pa
import pyarrow.parquet as pq

table = pa.Table.from_pandas(data)
pq.write_table(table, "troop_movements10m.parquet")

In [24]:
import pickle
import pandas as pd

with open("trained_model.pkl", "rb") as file:
    model = pickle.load(file)
    
data = pd.read_parquet("troop_movements10m.parquet")

rawX = data[["homeworld", "unit_type"]]
x = pd.get_dummies(rawX, columns=["homeworld", "unit_type"])
y_pred = model.predict(x)
print(y_pred[:10])

data["prediction"] = y_pred
print(data.head())

['resistance' 'resistance' 'empire' 'resistance' 'empire' 'empire'
 'empire' 'resistance' 'empire' 'resistance']
             timestamp  unit_id     unit_type  location_x  location_y  \
0  2023-06-13 17:33:18        1         at-st         2.0         8.0   
1  2023-06-13 17:33:17        2  tie_silencer         4.0         4.0   
2  2023-06-13 17:33:16        3         at-at         0.0         3.0   
3  2023-06-13 17:33:15        4  tie_silencer         6.0         1.0   
4  2023-06-13 17:33:14        5   tie_fighter         0.0         4.0   

   destination_x  destination_y    homeworld  prediction  
0              1              1  Glee Anselm  resistance  
1              0              1    Trandosha  resistance  
2              6              1     Corellia      empire  
3              6              9        Shili  resistance  
4              9              6   Muunilinst      empire  
